In [ ]:

from keras import models
from keras.models import load_model
# 这里为模型的地址
best_model = load_model(r'')
# best_model.summary()

out_put = layer_outputs = [layer.output for layer in best_model.layers[5:6]]
activation_model = models.Model(inputs=best_model.input, outputs=layer_outputs)


In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
train_dir = r''
validation_dir = r''
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 48, 10, 128))
    labels = np.zeros(shape=sample_count)
    generator = datagen.flow_from_directory(
                                            directory,
                                            target_size=(400, 100),
                                            batch_size=batch_size,
                                            class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = activation_model.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

# 这里下面要填训练集的验证集图片的总数
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)

train_features = np.reshape(train_features, (2000, 48*10*128))
validation_features = np.reshape(validation_features, (1000, 48*10*128))

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=48 * 10 * 128))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='softmax'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
loss='categorical_crossentropy',
metrics=['acc'])
history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels)
                    )

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
